In [1]:
import os 
import pandas as pd
from tqdm import tqdm
import numpy as np

from src.data_types import TOB, Order
from src.exchange import TOB_Exchange

%load_ext autoreload 
%autoreload 2

In [2]:
# define folder path
# folder_path = 'C:/Users/gobel/code/research_env/keyrock/bi-repo/data/tardis/datasets/'
folder_path = '/Users/jan-gobeli/Code/bi-repo/data/tardis/datasets/'
# folder_path = '/Users/nick-kokkaliaris/Desktop/Keyrock/Keyrock/Tardis Playground/datasets/'
files_list = [
    f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) 
    and f.split('_')[0] == 'binance' 
    and f.split('_')[-1] == 'COMPBTC.csv.gz'
    #and f.split('_')[-2] == 'OG'
    and f.split('_')[1] == 'quotes']
files_list

['binance_quotes_2023-07-02_COMPBTC.csv.gz',
 'binance_quotes_2023-07-05_COMPBTC.csv.gz',
 'binance_quotes_2023-07-04_COMPBTC.csv.gz',
 'binance_quotes_2023-07-03_COMPBTC.csv.gz']

In [3]:
# bin_trades = [
#     f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) 
#     and f.split('_')[0] == 'binance' 
#     and f.split('_')[-1] == 'COMPBTC.csv.gz'
#     #and f.split('_')[-2] == 'OG'
#     and f.split('_')[1] == 'trades']
# u_trades = [
#     f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) 
#     and f.split('_')[0] == 'upbit' 
#     and f.split('_')[-1] == 'BTC-COMP.csv.gz'
#     #and f.split('_')[-2] == 'OG'
#     and f.split('_')[1] == 'trades']
# ufiles_list = [
#     f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) 
#     and f.split('_')[0] == 'upbit' 
#     and f.split('_')[-1] == 'BTC-COMP.csv.gz'
#     #and f.split('_')[-2] == 'OG'
#     and f.split('_')[1] == 'quotes']

In [4]:
def get_data(files_list):
    df = pd.DataFrame()
    # Use a loop to get all the csv.gz files from the path location in a single Dataframe
    for csv_file in tqdm(files_list):
        file_name = (folder_path + csv_file)
        if file_name[-2:] == "gz":
            df1 = pd.read_csv(file_name, compression='gzip')
            # Concatenate all the dataframes into a single dataframe
            df = pd.concat([df, df1], ignore_index=True)
    return df

In [5]:
# upbit = get_data(ufiles_list)
binance = get_data(files_list)

# upbit_t = get_data(u_trades)
# binance_t = get_data(bin_trades)

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 16.41it/s]


In [6]:
binance['time'] = pd.to_datetime(binance.timestamp, unit='us')
# upbit['time'] = pd.to_datetime(upbit.timestamp, unit='us')
# binance_t['time'] = pd.to_datetime(binance_t.timestamp, unit='us')
# upbit_t['time'] = pd.to_datetime(upbit_t.timestamp, unit='us')

binance.set_index('time', inplace=True)
# upbit.set_index('time', inplace=True)
# binance_t.set_index('time', inplace=True)
# upbit_t.set_index('time', inplace=True)

In [7]:
# binance_t = binance_t[['timestamp', 'side', 'price', 'amount']].groupby(['timestamp', 'side', 'price']).sum().reset_index()

In [8]:
binance['latency'] = binance.local_timestamp - binance.timestamp
binance.latency.std() / binance.latency.mean()

0.2902754220552034

In [51]:
lst = binance[['timestamp', 'bid_amount', 'bid_price', 'ask_price', 'ask_amount']].sort_values('timestamp').to_numpy()

symbol = 'COMPBTC'

exchg = TOB_Exchange(latency=[5000, 0.29])
exchg.add_market(symbol, 'COMP', 'BTC')
exchg.add_balance('COMP', 0)
exchg.add_balance('BTC', 0.1)
exchg.load_tob(lst, symbol)

In [52]:
binance['spread'] = binance.ask_price / binance.bid_price - 1
binance['bid_pressure'] = binance.bid_amount / (binance.bid_amount + binance.ask_amount)

In [53]:
def signal(tob):
    bid_pressure = tob['bid_quantity'] / (tob['bid_quantity'] + tob['ask_quantity'])

    if bid_pressure >= 0.999: 
        return 1 
    
    elif bid_pressure <= 0.5: 
        return -1 
    
    else: 
        return 0

In [54]:
class tob_imbalance(): 
    def __init__(self):
        self.symbol = 'COMPBTC'
        self.amount = 1

        self.flag = False

    def run_strategy(self):

        tob = exchg.fetch_tob(self.symbol)

        decision = signal(tob)

        if (decision == 1) and not (self.flag): 
            exchg.market_order(symbol, self.amount, decision, tob['timestamp'])
            self.flag = True

        elif (decision == -1) and (self.flag): 
            exchg.market_order(symbol, self.amount, decision, tob['timestamp'])
            self.flag = False

        else: 
            pass

In [55]:
exchg.run_simulation(tob_imbalance)

Trade Executed Trade(trade_id=199, order_id=201, side=1, taker=True, amount=1, price=0.001716, entryTime=1688256004133884.0, eventTime=1688256004133884.0)
Trade Executed Trade(trade_id=200, order_id=202, side=-1, taker=True, amount=1, price=0.001678, entryTime=1688264792960093.5, eventTime=1688264792960093.5)
Trade Executed Trade(trade_id=201, order_id=203, side=1, taker=True, amount=1, price=0.001678, entryTime=1688264793160929.8, eventTime=1688264793160929.8)
Trade Executed Trade(trade_id=202, order_id=204, side=-1, taker=True, amount=1, price=0.001813, entryTime=1688289151271236.8, eventTime=1688289151271236.8)
Trade Executed Trade(trade_id=203, order_id=205, side=1, taker=True, amount=1, price=0.001812, entryTime=1688289155472195.0, eventTime=1688289155472195.0)
Trade Executed Trade(trade_id=204, order_id=206, side=-1, taker=True, amount=1, price=0.00181, entryTime=1688290431464100.5, eventTime=1688290431464100.5)
Trade Executed Trade(trade_id=205, order_id=207, side=1, taker=True,

In [56]:
exchg.trades

SortedDict({1688256004133884.0: Trade(trade_id=199, order_id=201, side=1, taker=True, amount=1, price=0.001716, entryTime=1688256004133884.0, eventTime=1688256004133884.0), 1688264792960093.5: Trade(trade_id=200, order_id=202, side=-1, taker=True, amount=1, price=0.001678, entryTime=1688264792960093.5, eventTime=1688264792960093.5), 1688264793160929.8: Trade(trade_id=201, order_id=203, side=1, taker=True, amount=1, price=0.001678, entryTime=1688264793160929.8, eventTime=1688264793160929.8), 1688289151271236.8: Trade(trade_id=202, order_id=204, side=-1, taker=True, amount=1, price=0.001813, entryTime=1688289151271236.8, eventTime=1688289151271236.8), 1688289155472195.0: Trade(trade_id=203, order_id=205, side=1, taker=True, amount=1, price=0.001812, entryTime=1688289155472195.0, eventTime=1688289155472195.0), 1688290431464100.5: Trade(trade_id=204, order_id=206, side=-1, taker=True, amount=1, price=0.00181, entryTime=1688290431464100.5, eventTime=1688290431464100.5), 1688290432067282.2: 

In [57]:
exchg.balance

{'COMP': -97, 'BTC': 0.29521803580000006}

In [14]:
exchg.limit_order('COMPBTC', 20, 0.001712, 1)
exchg.limit_order('COMPBTC', 10, 0.001718, 0)

In [35]:
for i in range(5):
    exchg._simulation_step()

Bid: 0.001717 | 0.001718 :Ask
Open Buy Orders
0.001712 @ 20
Open Sell Orders
Bid: 0.001717 | 0.001718 :Ask
Open Buy Orders
0.001712 @ 20
Open Sell Orders
Bid: 0.001717 | 0.001718 :Ask
Open Buy Orders
0.001712 @ 20
Open Sell Orders
Bid: 0.001717 | 0.001718 :Ask
Open Buy Orders
0.001712 @ 20
Open Sell Orders
Bid: 0.001717 | 0.001718 :Ask
Open Buy Orders
0.001712 @ 20
Open Sell Orders
